# **Final Exam** 
### *Tan Yu*
***
`Here are some functions:`
####  1.  *Import and preprocess the data*

In [4]:
import csv
import pandas
import numpy as np
import operator
xtest = pandas.read_csv('bbb.test.csv', header = None)
xtrain = pandas.read_csv('bbb.train.csv',header = None)
ytest = pandas.read_csv('labels.test.csv',header = None)
ytrain = pandas.read_csv('labels.train.csv',header = None)
yytrain = ytrain.replace('nc', 1)
fytrain = yytrain.replace('c', 0)
yytest = ytest.replace('nc', 1)
fytest = yytest.replace('c', 0)

#### 2. *The Confusion Matrix Generator Function*

In [21]:
def confusionM(truey, prey):
    v00, v01, v10, v11 = 0, 0, 0, 0
    for i in range(len(truey)):
        if truey[i] == 0:
            if prey[i] == 0:
                v00 += 1
            else:
                v10 += 1
        else:
            if prey[i] == 0:
                v01 += 1
            else:
                v11 += 1
    print('Confusion Matrix|', 'True c(0)', 'True nc(1)', 'Total')
    print('  Predicc c(0)  |',  v00,'       ', v01,'       ', v00+v01)
    print('  Predicc nc(1) |',  v10,'      ', v11,'      ', v10+v11)
    print('  Total         |',  v10+v00,'      ', v11+v01,'      ', v10+v11+v00+v01)

#### 3.*The prediction function*

In [9]:
def PredictFunc(x, y, beta):
    res = {}
    res['error'] = 0
    res['prey'] = []
    res['truey'] = []
    for i in range(x.shape[0]):
        temp = x.iloc[i,:]
        tempty = y.iloc[i,0]
        temppy = PredOneRow(temp, beta)
        res['prey'].append(temppy)
        res['truey'].append(tempty)
        if tempty != temppy:  
            res['error'] += 1
    res['accu'] = 1 - res['error'] / x.shape[0]
    res['mse'] = res['error'] / x.shape[0]
    return res

def PredOneRow(x, beta):
    x = np.hstack((np.ones((1)), x.T))
    pre = pi_val(np.dot(x, beta))
    if pre > 0.5:
        return 1
    return 0

#### 4.*The function to obtain $\pi_i$*

In [7]:
def pi_val(input):
    output = 1 / (1 + np.exp(-input))
    return output

***
## Problem (a) : *Log-likelihood function*
\begin{align*}
l(\beta) & = \log(L(\beta)) = \log(\prod_{i = 1}^{p} \pi_i^{y_i}(1-\pi_i)^{1 - y_i}) = \sum_{i = 1}^p y_i \log(\pi_i) + (1-y_i)\log(1-\pi_i) \\ & = \sum_{i = 1}^{p} (y_i \log(\dfrac{e^{\beta_0 + \beta^Tx}}{1+e^{\beta_0 + \beta^Tx}}) + (1-y_i)\log(\dfrac{1}{1+e^{\beta_0 + \beta^Tx}})) \\& = \sum_{i = 1}^p y_i(\beta_0 + \beta^Tx) - \log(1 + e^{\beta_0 + \beta^Tx})
\end{align*}
***

## Problem (b) : *Logistic Regression*

### *(1) The gradient*
The first-order partial derivatives of the log-likelihood for each $\beta_k, k = 0, 1, 2, ...,n$:
\begin{align*}
\dfrac{\partial l(\beta)}{\partial \beta_k} = \sum_{i=1}^p y_i x_{ik} - \pi_i x_{ik} = \sum_{i = 1}^p x_{ik}(y_i - \pi_i)
\end{align*}

### *(2) Run the model*

In [27]:
# Logistic Regression
def GradientD(x, y, stepsize, iters):
    beta = np.random.random_sample((x.shape[1],1))
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        ########################
        grad = np.dot(x.T, y - yy)
        beta = beta - stepsize*grad
    return beta

def LR(x, y, stepsize, iters):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD(x, y, stepsize, iters)
    return beta

def OutputFuncLo(beta, res):
    print('Coefficient: ', beta.flatten(), '\n')
    print('Error Num:', res['error'], '  Accuracy:', res['accu'], '\n')

betaLo = LR(xtrain, fytrain, 0.01, 200)
resLo = PredictFunc(xtest, fytest, betaLo)
OutputFuncLo(betaLo, resLo)
confusionM(resLo['truey'], resLo['prey'])

Coefficient:  [193.59324301 139.76977672  63.93317386  10.39628063  37.53354754
  -4.08151957  53.50637824 -11.00744558  60.29613406  23.58543401
  64.52870379  15.0074556   56.56387753  -3.67342168  31.13642746
  -9.14865753  60.34606201 -10.11079168  49.58187773   1.72000861
  26.52278717  22.90649261  24.00779042  -5.20985243  37.98660774
  -0.68047677 126.45654626 -12.76578706  36.52087995  -5.20613991
  18.99514317  11.67972428  30.00025865  16.08483339] 

Error Num: 27   Accuracy: 0.619718309859155 

Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 0         0         0
  Predicc nc(1) | 27        44        71
  Total         | 27        44        71


***

## Problem (c) : *Ridge Regression*
### (1) *Cost Function* 
\begin{align*}
J(\beta) = \sum_{i=1}^p (y_i - x_i^T \beta)^2 + \lambda \left\| \beta \right\|_2^2 \end{align*}

### (2) *Explain why we use Ridge regression*

### (3) *Run the model*

 \begin{align*}\text{Gradient of Ridge Regression}:\dfrac{\partial J(\beta)}{\partial \beta_k} = -2\sum_{i=1}^p x_{ik}(y_i-\pi_i) + \lambda \beta_k\end{align*}
 

In [34]:
def GradientD3(x, y, stepsize, iters, lam3):
    #beta = np.ones((x.shape[1],1))
    beta = np.random.random_sample((x.shape[1],1)) # 34*1
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = -2*np.dot(x.T, y - yy)+ lam3 * beta
        #print(grad)
        beta = beta - stepsize*grad
    return beta

def RR(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD3(x, y, stepsize, iters, lam3)
    return beta

def cvkfold(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = RR(xtrain_train, ytrain_train, 0.01, 100, lam3)
            restemp = PredictFunc(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse
 
def usebestlam(lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = 0
    for i in range(len(lambdalist)):
        if lambdalist[i] <= up:
            up = lambdalist[i]
            index = i
    bestlam = lambdapotential[index]
    beta1 = RR(xtrain, fytrain, 0.01, 100, bestlam)
    res = PredictFunc(xtest, fytest, beta1)
    print('Coefficient: ', beta1.flatten(), '\n')
    print('Accuracy: ', res['accu'], ' Best Lambda: ', bestlam, '\n')
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

lambdapotential = [0.03, 0.01, 10]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold(data, lambdapotential, 5)
#print(lambdalist)
acc, lam = usebestlam(lambdapotential, xtrain, fytrain, xtest, fytest)

Coefficient:  [-5.28735203  3.40603659  2.09529647  0.29911712  3.4043022   1.48010192
  0.45004933  2.91646998  2.0345149  -0.20697624 -1.69097149 -1.30529714
 -0.87222543  1.44371649  1.75939505  0.21519688  0.37980811  1.83126508
 -0.92971087 -0.56015074  0.42888689 -3.51177069  2.62428285  1.62463815
  1.53425564  1.19754346 -3.50851375 -0.84882865  1.41354569  1.60194299
  0.26196949 -0.66868446  0.08900893 -2.83205961] 

Accuracy:  0.8309859154929577  Best Lambda:  0.01 

Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 15         0         15
  Predicc nc(1) | 12        44        56
  Total         | 27        44        71


***

## Problem (d) : *LASSO Regression*
### (1) *Cost Function* 
\begin{align*}
J(\beta) = \sum_{i=1}^p (y_i - \sum_{j=1}^nx_{ij} \beta_j)^2 + \lambda \sum_{j=1}^n\left| \beta_j \right| \end{align*}

### (2) *Explain why we use Lasso regression*

### (3) *Run the model*
1. Randomly set $\beta^0$ for iteration 0
2. For $k$th iteration:
   * Compute gradient $\nabla f(\beta^{k-1})$
   * Set $z = \beta^{k-1} - \frac{1}{\text{step size}} \nabla f(\beta^{k-1})$
   * $\beta^k = sgn(z) \max[\left|z\right|-\frac{\lambda}{\text{step size}}, 0]$
 
***

## Problem (e) : *Best Model*